In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='gemma2-9b-it')

In [ ]:
llm.invoke("How will the weather be in munich today?")

AIMessage(content='I do not have access to real-time information, including weather forecasts. To get the current weather conditions and forecast for Munich, I recommend checking a reliable weather app or website.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 18, 'total_tokens': 57, 'completion_time': 0.070909091, 'prompt_time': 0.002216007, 'queue_time': 0.25106092199999996, 'total_time': 0.073125098}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--43909d55-473e-4411-97f5-bedd4bb5e040-0', usage_metadata={'input_tokens': 18, 'output_tokens': 39, 'total_tokens': 57})

In [3]:
from langchain_core.tools import tool

@tool
def get_weather(loc: str):
    """Call to get current weather."""
    if loc.lower() in ['mumbai']:
        return "Its 15 degrees."
    else:
        return "Its 30 degrees."

@tool
def check_seating_availability(loc: str, type: str):
    """Call to check seating availability."""
    if loc.lower() == 'mumbai' and type == 'outdoor':
        return "Yes we have seats."
    elif loc == 'mumbai' and type == 'indoor':
        return "Yes we have indoor seats."
    else:
        return "No seats."

tools = [get_weather, check_seating_availability]

llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke('Will i get indoor seats in mumbai?')
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hq3x', 'function': {'arguments': '{"loc":"mumbai","type":"indoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 1087, 'total_tokens': 1180, 'completion_time': 0.169090909, 'prompt_time': 0.047306249, 'queue_time': 0.260217371, 'total_time': 0.216397158}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8718774a-0614-4bd3-accd-1f0f7009675a-0', tool_calls=[{'name': 'check_seating_availability', 'args': {'loc': 'mumbai', 'type': 'indoor'}, 'id': 'call_hq3x', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1087, 'output_tokens': 93, 'total_tokens': 1180})

In [4]:
result.tool_calls

[{'name': 'check_seating_availability',
  'args': {'loc': 'mumbai', 'type': 'indoor'},
  'id': 'call_hq3x',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

In [ ]:
result.tool_calls

In [5]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6am0', 'function': {'arguments': '{"loc":"munich"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 1096, 'total_tokens': 1179, 'completion_time': 0.150909091, 'prompt_time': 0.038253801, 'queue_time': 0.257068028, 'total_time': 0.189162892}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4d8d65db-2d87-4df4-90ec-890418591a9b-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'munich'}, 'id': 'call_6am0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1096, 'output_tokens': 83, 'total_tokens': 1179})

In [6]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'loc': 'munich'},
  'id': 'call_6am0',
  'type': 'tool_call'}]

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [ ]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [ ]:
llm_output.tool_calls

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)